# Train Character Language Model
Language Models are used for the following tasks:
* Check the probability of a sentence to be part of some specific language
* Generate Words/Characters given a sentence
On this notebook we will train a Neural Character Language model based on GRUs.
![Input Output](docs/imgs/char_lang_model.png "Title")

#### Evaluation Metrics
Character Language models are evaluated with Character Error Rate metric, that can be calculated by doing the exp of the cross entropy loss

#### Datasets
* https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/

#### References:
* https://en.wikipedia.org/wiki/Language_model
* https://github.com/furkanu/deeplearning.ai-pytorch
* https://towardsdatascience.com/writing-like-shakespeare-with-machine-learning-in-pytorch-d77f851d910c
* https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html
* https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import time
import random
import os

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torch.utils.data as utils
from torch.optim import lr_scheduler
import torch.distributions

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from tqdm import tqdm
import utils_char_dataset
import utils_char_lm
import model as models
from char_LM_dataset import CharacterLanguageModelDataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
print('Device:', device)
num_gpu = torch.cuda.device_count()
num_gpu = 1
print('Number of GPUs Available:', num_gpu)

pickle_filename_train = "data/shakespeare_corpus_data_train.pkl"
pickle_filename_test = "data/shakespeare_corpus_data_test.pkl"
codemap = utils_char_dataset.load_pickle('./codemap_LM.pickle')
num_classes = len(codemap)
print('Num classes:', num_classes)

# Hyperparameters
# Pure sequence to sequence models can't deal with batches
batch_size = 200
clip = 50.0
lr = 0.001
hidden_size = 256 #256
num_layers = 2 #3
epochs = 150

Device: cuda:0
Number of GPUs Available: 1
Num classes: 69


#### Load Data

In [2]:
char_dataset_train = CharacterLanguageModelDataset(pickle_filename_train)
dataloader_train = utils.DataLoader(char_dataset_train, batch_size=batch_size, shuffle=True)
char_dataset_test = CharacterLanguageModelDataset(pickle_filename_test)
dataloader_test = utils.DataLoader(char_dataset_test, batch_size=batch_size, shuffle=False)
sample = char_dataset_train[0]
sample_str = ''.join([utils_char_dataset.char_from_class_id(char, codemap) for char in sample['X'][0:sample['len_x']]])
print(sample_str)

in


#### Start Tensorboard Interface

In [3]:
#writer = SummaryWriter('./logs')
# Default directory "runs"
writer = SummaryWriter()

#### Define Masked Loss
We need to filter out from the gradient the padding values.
##### Parameters
* input(Decoder output): [batch x sequence]
* target(Label): [batch]
* mask shape: [batch]

##### Example
```python
input: torch.Size([32, 83])
target: torch.Size([32])
mask: torch.Size([32])

input: torch.Size([70, 84, 69])
target: torch.Size([70, 84])
mask: torch.Size([70, 84])
```

In [4]:
def maskNLLLoss(predicted, target, mask):    
    # Get total number of valid elements
    nTotal = mask.sum()
    
    # flatten all the labels, mask and prediction
    target = target.view(-1)
    mask = mask.view(-1)
    
    predicted = predicted.view(-1, num_classes)        
    
    # pick the values for the label and zero out the rest with the mask
    #predicted_mask = predicted[range(predicted.shape[0]), target] * mask
    predicted_mask = -torch.log(predicted[range(predicted.shape[0]), target])
    loss = predicted_mask.masked_select(mask).mean()        
        
    loss = loss.to(device)    
    return loss, nTotal.item()

In [5]:
def evaluate_test(seq_model, writer = None):
    
    seq_model.eval()

    # Iterate on the test set
    print_every = 10
    metrics = []
    for iter, sample in enumerate(dataloader_test):
        input_tensor = sample['X'].type(torch.LongTensor).to(device)
        target_tensor = sample['Y'].type(torch.LongTensor).to(device)
        target_mask = sample['label_mask'].type(torch.ByteTensor).to(device)
        len_input = sample['len_x'].to(device)
        len_target = sample['len_y'].to(device)
        curr_batch_size = len_input.shape[0]
        with torch.no_grad():
            hidden_state = models.initHidden(curr_batch_size, False, hidden_size, num_layers, device)
            # Run words through seq_model (all batch at once)   
            seq_model_outputs, hidden_state = seq_model(input_tensor, hidden_state, len_input)        

            loss, nTotal = maskNLLLoss(seq_model_outputs, target_tensor, target_mask)
            
            # Add predicted/target text to tensorboard
            if writer:
                pass

            metric = loss.item()
            
            
            metrics.append(metric)

    distance_test = np.mean(metrics)
    return distance_test

In [6]:
def train_batch(input_tensor, target_tensor, len_input, len_target, target_mask, seq_model,  
          model_optimizer, iterations):
    # Start seq_model hidden state as zero
    curr_batch_size = len_input.shape[0]
    
    hidden_state = models.initHidden(curr_batch_size, False, hidden_size, num_layers, device)

    # Zero the gradient for doing backprop
    model_optimizer.zero_grad()  

    # Initialize Loss
    loss = 0
    print_losses = []
    n_totals = 0
        
    # Run words through seq_model (all batch at once)   
    seq_model_outputs, hidden_state = seq_model(input_tensor, hidden_state, len_input)        
    
    loss, nTotal = maskNLLLoss(seq_model_outputs, target_tensor, target_mask)    
    
    # Calculate the loss gradient wrt to the model weights
    loss.backward()
    
    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(seq_model.parameters(), clip)    

    # Do the gradient descent step
    model_optimizer.step()    

    # Return normalized loss
    return loss.item()

In [7]:
def train_loop(seq_model, n_epochs=100, learning_rate=0.01):

    # Initialize SGD Optimizer to train the network
    seq_model_optimizer = optim.Adam(seq_model.parameters(), lr=learning_rate)    
    sc_plt_enc = torch.optim.lr_scheduler.ReduceLROnPlateau(seq_model_optimizer, patience=4, verbose=True)    
        
    iterations = 0
    best_metric = 1e12
    for epoch in range(n_epochs):
        running_loss = 0.0
        seq_model.train()        
        for iter, sample in enumerate(dataloader_train):
            # Select a sample and transpose padded arrays into (max_len_sequence x batch_size)            
            
            # Each 10 iterations send some input data to tensorboard
            if iterations % 10 == 0:
                pass
                # Select first element from the Batch and send to tensorboard
                #stroke = sample['sequence'][0].numpy()
                #lbl_str = sample['label_str'][0]         
                #fig_input = iam.line_plot(stroke, lbl_str, display=False)
                #writer.add_figure('train/stroke_target', fig_input, iterations)
        
            #input_tensor = sample['X'].type(torch.FloatTensor).to(device)
            input_tensor = sample['X'].type(torch.LongTensor).to(device)
            target_tensor = sample['Y'].type(torch.LongTensor).to(device)
            target_mask = sample['label_mask'].type(torch.ByteTensor).to(device)
            len_input = sample['len_x'].to(device)
            len_target = sample['len_y'].to(device)                        
                
            # Train on that particular input/output sequence
            loss = train_batch(input_tensor, target_tensor, len_input, len_target, target_mask, 
                         seq_model, seq_model_optimizer, iterations)

            # Accumulate Loss for display
            running_loss += loss

            # Send loss to Tensorboard
            #print(loss)
            writer.add_scalar('train/loss', loss, iterations)
            iterations +=1
        
        # Epoch ended        
        sc_plt_enc.step(running_loss)        
        writer.add_scalar('train/running_loss', running_loss, epoch)
        
        # Get current learning rate (To display on Tensorboard)
        for param_group in seq_model_optimizer.param_groups:
            curr_learning_rate = param_group['lr']
            writer.add_scalar('train/learning_rate', curr_learning_rate, epoch)
        
        # Evaluate Model
        metric = evaluate_test(seq_model, writer)
        writer.add_scalar('test/metric_loss', metric, epoch)
        
        test_metric = metric 
        #print('Epoch:', epoch, 'Running loss:', running_loss)
        
        # Save model on the best evaluation distance 
        if test_metric < best_metric:
                print('Smallest metric at epoch:', epoch, 'metric:', test_metric, 'running_loss:', running_loss)
                torch.save({'seq_model': seq_model.state_dict(),}, 
                           os.path.join('./', '{}_{}.pt'.format('model_lm', 'best')))
                best_metric = test_metric

In [ ]:
# Add to tensorboard hyper-parameter values
writer.add_text('train/params', 
                'learning_rate:' + str(lr) + 
                ' hidden_size:' + str(hidden_size) + 
                ' num_layers:' + str(num_layers), 0)

# Instantiate Neural Character Language Model Networks
char_LM = models.CharLangModel(num_classes, hidden_size, num_classes, num_layers=num_layers).to(device)

# Train
train_loop(char_LM, n_epochs=epochs, learning_rate=lr)

Smallest metric at epoch: 0 metric: 1.126661451190644 running_loss: 5218.216045677662
Smallest metric at epoch: 1 metric: 1.0934671594404064 running_loss: 4478.075229525566
Smallest metric at epoch: 2 metric: 1.0806190542001153 running_loss: 4387.168428063393
Smallest metric at epoch: 3 metric: 1.0749600771525483 running_loss: 4344.22549033165
Smallest metric at epoch: 4 metric: 1.0704284019586516 running_loss: 4318.8025180101395
Smallest metric at epoch: 5 metric: 1.0683795985520017 running_loss: 4302.5816569924355
Smallest metric at epoch: 6 metric: 1.0659847008415442 running_loss: 4291.340185821056
Smallest metric at epoch: 7 metric: 1.0643659338454186 running_loss: 4283.761633038521


#### Get the Probability of a Sentence

In [ ]:
lst_words = ['Thi1', 'This', 'Love', 'Lov1', 'Hellz', 'Hello', 'HellO', 'HeLlo']
for word in lst_words:
    print('P(%s):%f' % (word, utils_char_lm.getProbabilitySentence(word, char_LM, device, codemap)))

#### Generate Characters Given some chars
##### References:
* https://pytorch.org/docs/stable/distributions.html

##### Greedly

In [ ]:
lst_words = ['t', 'k', 'de', 'Liv', 'Pre', 'To be or']
for word in lst_words:
    pred = utils_char_lm.getNextChar(word,100, char_LM, device, codemap)
    res_str = ''.join([utils_char.char_from_class_id(class_id, codemap) for class_id in pred])
    print('('+word+')'+res_str)

##### Sampled

In [ ]:
lst_words = ['t', 'k', 'de', 'Liv', 'Pre', 'To be or']
for word in lst_words:
    pred = utils_char_lm.getNextChar(word,100, char_LM, device, codemap, greedly=False)
    res_str = ''.join([utils_char.char_from_class_id(class_id, codemap) for class_id in pred])
    print('('+word+')'+res_str)